In [1]:
import pandas as pd
import numpy as np
from itertools import chain

In [2]:
df = pd.read_csv('prepost_matched_debt_files.csv', index_col = 0)

In [3]:
def stringToList(x):
    lst = x.strip('][\"\'').split(', ') if not pd.isnull(x) else x
    return [x.strip('\"\' ') for x in lst] if not pd.isnull(x) else x

In [4]:
df['full name'] = df['full name'].apply(lambda x: stringToList(x))
df['Same State Loan Office'] = df['Same State Loan Office'].apply(lambda x: 
                                                                  stringToList(x))
df['Different State Loan Office'] = df['Different State Loan Office'].apply(lambda x: 
                                                                            stringToList(x))
df['Same State Liquidated Debt'] = df['Same State Liquidated Debt'].apply(lambda x: 
                                                                          stringToList(x))
df['Different State Liquidated Debt'] = df['Different State Liquidated Debt'].apply(lambda x: 
                                                                                    stringToList(x))
df['Pierce Certificates'] = df['Pierce Certificates'].apply(lambda x: 
                                                            stringToList(x))

# How many unique individuals were issued 6 percent stocks and deferred 6 percent stocks in 1790 and after?

In [5]:
df.groupby('state')['full name'].count()

state
BM       2
BVI      4
CT     684
DE       7
FR       2
GA      14
GB       2
MA     108
MD     277
NC      41
NH     132
NJ      32
NY      39
PA     609
RI     335
SC     208
US       1
VA      27
VI       4
VT       6
Name: full name, dtype: int64

# How many unique groups of individuals
- were original purchasers of loan office certicates of the same state as the 6 percent stock?
- were original purchasers of loan office certicates issued from another state?
- were original recipients of liquidated debt certicates issued by the same-state loan office? other state loan offices?
- were original recipients of the Pierce Certicates?

### How many unique groups of individuals were original purchasers of loan office certicates of the same state as the 6 percent stock?

In [6]:
df[df['Same State Loan Office'].apply(lambda x: 
                                      type(x) == list)]['state'].value_counts()

CT    304
PA    241
MD     92
NH     53
MA     34
SC     14
RI     11
NJ     11
VA     10
NY      4
DE      2
GA      2
Name: state, dtype: int64

### How many unique groups of individuals were original purchasers of loan office certicates of a different state as the 6 percent stock?

In [7]:
df[df['Different State Loan Office'].apply(lambda x: 
                                           type(x) == list)]['state'].value_counts()

CT    98
PA    97
MD    53
MA    36
NH    25
VA    15
SC    15
NY    14
NJ    11
RI     8
DE     6
GA     2
Name: state, dtype: int64

##### Aside: How many unique groups of individuals were original purchasers of loan office certicates of a different and same state as the 6 percent stock?

In [8]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Loan Office'], 
                  df['Different State Loan Office'])]]['state'].value_counts()

PA    74
CT    59
MD    24
MA    22
SC    11
VA     9
NJ     7
RI     7
NH     6
NY     3
GA     2
DE     2
Name: state, dtype: int64

### How many unique groups of individuals were original recipients of liquidated debt certificates issued by the same state loan office?

In [9]:
df[df['Same State Liquidated Debt'].apply(lambda x: 
                                          type(x) == list)]['state'].value_counts()

PA    173
CT    111
MA     27
NH     26
NJ     10
SC      7
RI      7
MD      7
NY      6
VA      2
GA      2
DE      1
NC      1
Name: state, dtype: int64

### How many unique groups of individuals were original recipients of liquidated debt certificates issued by another state loan office?

In [10]:
df[df['Different State Liquidated Debt'].apply(lambda x: 
                                               type(x) == list)]['state'].value_counts()

CT    174
PA     98
NH     43
MA     38
MD     38
RI     13
SC     12
NJ     11
NY      8
DE      2
GA      2
VA      1
Name: state, dtype: int64

##### Aside: How many unique groups of individuals were original purchasers of liquidated debt certificates from the same and another loan office?

In [11]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Liquidated Debt'], 
                  df['Different State Liquidated Debt'])]]['state'].value_counts()

CT    110
PA     55
NH     26
MA     25
NJ     10
RI      7
SC      6
MD      6
NY      5
GA      2
DE      1
Name: state, dtype: int64

### How many unique groups of individuals were original recipients of the Pierce Certicates, corresponding to the same state?

In [12]:
df[df['Pierce Certificates'].apply(lambda x: 
                                   type(x) == list)]['state'].value_counts()

CT    169
PA    131
MD     65
MA     32
NH     25
SC     14
RI     14
NY     13
VA      7
NJ      6
DE      3
GA      2
NC      1
Name: state, dtype: int64

### For how many unique groups of individuals do the face values of their pre-1790 certificates equal the sum of the face values of the 6 percent and deferred 6 percent stocks that they received?

##### Number of people with non-zero assets and that the debt values from pre-1790 and post-1790 are equal

In [13]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]

,full name,state,state debt,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
36,[Aaron Kelsey],CT,CT,[Aaron Kelsey],NaN,NaN,NaN,NaN,95.10,95.10834,0.0,0.000000,0.0,0.0,95.108340,1
45,[Abel Morse],CT,CT,[Abel Morse],NaN,NaN,NaN,NaN,1388.83,1388.86930,0.0,0.000000,0.0,0.0,1388.869300,1
46,[Abel Patchen],CT,CT,[Abel Patchen],NaN,NaN,NaN,NaN,153.96,153.97916,0.0,0.000000,0.0,0.0,153.979160,1
68,[Abraham Brinsmade],CT,CT,[Abraham Brinsmade],NaN,NaN,NaN,NaN,400.00,400.00000,0.0,0.000000,0.0,0.0,400.000000,1
121,[Amos Miller],CT,CT,[Amos Miller],NaN,NaN,NaN,NaN,654.05,654.07640,0.0,0.000000,0.0,0.0,654.076400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251,[Thomas Hawkins],MD,MD,[Thomas Hawkins],NaN,NaN,NaN,NaN,120.72,120.72224,0.0,0.000000,0.0,0.0,120.722240,1
2387,[William Dickensheet],MD,MD,[William Dickensheet],NaN,NaN,NaN,NaN,50.00,50.00833,0.0,0.000000,0.0,0.0,50.008330,1
2427,[William House],MD,MD,[William House],NaN,NaN,NaN,NaN,25.47,25.48750,0.0,0.000000,0.0,0.0,25.487500,1
2484,[William Sitgreaves],PA,PA,NaN,NaN,[William Sitgreaves],NaN,NaN,1277.57,0.00000,0.0,1277.577778,0.0,0.0,1277.577778,1


In [14]:
df[df['tot_pre1790_certs'] != 0].shape[0]

1210

In [15]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

74

##### State Distribution

In [16]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

CT    45
MD    15
PA     9
NH     4
VA     1
Name: state, dtype: int64

##### Number of people with zero assets and equal values
Most of these occur when someone has 3% but no 6% certificates

In [17]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

11

##### State Distribution

In [18]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

CT    6
SC    3
PA    1
NY    1
Name: state, dtype: int64

### Who, and how many unique groups of people redeemed certificates with Hamilton and were not original owners/purchasers

In [19]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]].shape[0]

1313

In [20]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]

,full name,state,state debt,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
3,[James O'Hara],PA,PA,NaN,NaN,NaN,NaN,NaN,33.44,0.0,0.0,0.0,0.0,0.0,0.0,0
4,[Agness ann],PA,PA,NaN,NaN,NaN,NaN,NaN,883.63,0.0,0.0,0.0,0.0,0.0,0.0,0
5,[Anne Maria],PA,PA,NaN,NaN,NaN,NaN,NaN,255.77,0.0,0.0,0.0,0.0,0.0,0.0,0
6,[Charles Godfreid],PA,PA,NaN,NaN,NaN,NaN,NaN,3260.83,0.0,0.0,0.0,0.0,0.0,0.0,0
7,[Daniel W],PA,PA,NaN,NaN,NaN,NaN,NaN,269.57,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,[Zebulon Waterman],CT,CT,NaN,NaN,NaN,NaN,NaN,306.61,0.0,0.0,0.0,0.0,0.0,0.0,0
2530,[Zephaniah Andrews],RI,RI,NaN,NaN,NaN,NaN,NaN,1728.74,0.0,0.0,0.0,0.0,0.0,0.0,0
2531,[Zephaniah Brown],RI,RI,NaN,NaN,NaN,NaN,NaN,2415.08,0.0,0.0,0.0,0.0,0.0,0.0,0
2532,[Zephaniah Davis],CT,CT,NaN,NaN,NaN,NaN,NaN,77.20,0.0,0.0,0.0,0.0,0.0,0.0,0


##### state distribution

In [21]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['state'].value_counts()

RI     318
PA     248
CT     236
SC     186
MD     129
NH      43
MA      41
NC      40
NY      14
NJ      13
GA      12
VA      11
VT       6
VI       4
BVI      4
GB       2
FR       2
BM       2
US       1
DE       1
Name: state, dtype: int64

In [22]:
namelsts = df[[tot == 0 and tt != 0 for 
               tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['full name'].apply(lambda x: pd.Series(x)).values
names = [x for x in list(chain.from_iterable(namelsts)) if not pd.isnull(x)]

In [23]:
name_state_asset = pd.DataFrame({}, columns = ['Full Name', 'Total', 'state'])

In [24]:
name_df = df[[tot == 0 and tt != 0 for 
              tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['full name'].apply(lambda x: pd.Series(x))
name_df
for name in names:
    ind1 = name_df[name_df[0] == name].index.tolist()
    ind2 = name_df[name_df[1] == name].index.tolist()
    ind3 = name_df[name_df[2] == name].index.tolist()
    ind4 = name_df[name_df[3] == name].index.tolist()
    ind5 = name_df[name_df[4] == name].index.tolist()
    ind1.extend(ind2)
    ind1.extend(ind3)
    ind1.extend(ind4)
    ind1.extend(ind5)
    ind1 = list(set(ind1))
    tot_assets = df.loc[ind1, 'Total'].sum()
    state = df.loc[ind1, 'state'].values[0]
    add_df = pd.DataFrame()
    add_df['Full Name'] = [name]
    add_df['Total'] = [tot_assets]
    add_df['state'] = [state]
    name_state_asset = pd.concat([name_state_asset, add_df])

In [25]:
cd_names = pd.DataFrame(name_state_asset.groupby(['Full Name', 'state'])['Total'].sum()).reset_index()

### How many unique people redeemed certificates with Hamilton and were not original owners/purchasers

In [27]:
len(set(cd_names['Full Name']))

1325

In [28]:
cd_names.to_csv("speculator_names.csv")